# Data Wrangling - Capstone 2

## Content

All the following steps are being performed:
1. Data collection
2. Data organization
3. Data definition
4. Data cleaning

## Imports

In [1]:
import pandas as pd
import numpy
import glob
import warnings
import re

## 1. Data Collection

### 1.1 Load the data

In [2]:
# get the list of files in the data folder

file_names = glob.glob('data-FFF/Invest+Your+Values+shareclass+results+*.xlsx')
file_names.sort()
#file_names

### 1.2 Merge the data

In [3]:
# Using all files to generate one single file
# get the time from the file name and insert it into the dataframe as a column
data = None
warnings.filterwarnings("ignore", message="Unknown extension is not supported and will be removed")

for file in file_names:
    df = pd.read_excel(file, sheet_name = 'Shareclasses')
    print(file)
    df['current time'] = file[-13:-5]
    if data is None:
        data = df
    else:
        data = pd.concat([data, df])  

data-FFF/Invest+Your+Values+shareclass+results+20200414.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20200519.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20200610.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20200716.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20200811.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20200913.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20200928.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20201018.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20201111.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20210121.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20210303.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20210401.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20210503.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20210715.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20210809.xlsx
data-FFF/Invest+Your+Values+shareclass+results+20210902.xlsx
data-FFF/Invest+Your+Val

In [4]:
data.shape

(356518, 146)

In [5]:
type(data)

pandas.core.frame.DataFrame

### 1.3 Save the merged data

In [6]:
data.to_csv('data/all_results_data.csv', index = False)

In [7]:
# save dtype of each column
datatypes  = data.dtypes.to_frame('dtypes').reset_index()
type(datatypes)

pandas.core.frame.DataFrame

In [8]:
datatypes.head()

,index,dtypes
0,Fund profile: Shareclass name,object
1,Fund profile: Ticker,object
2,Fund profile: Fund name,object
3,Fund profile: Asset manager,object
4,Fund profile: Shareclass type,object


In [9]:
datatypes.to_csv('data/datatypes.csv', index = False)

## 2. Data Organization

### 2.1 Load the saved data

In [10]:
types = pd.read_csv('data/datatypes.csv')
dtypes_dict = types.set_index('index')['dtypes'].to_dict()
#dtypes_dict

In [11]:
df = pd.read_csv('data/all_results_data.csv', dtype = dtypes_dict)
#set(df['current time'])

In [12]:
fund_names = df['Fund profile: Fund name']
len(fund_names)

356518

In [13]:
len(set(fund_names))

4689

In [14]:
shareclasses = df['Fund profile: Shareclass name']
len(shareclasses)

356518

In [15]:
len(set(shareclasses))

14075

### 2.2 Clean the fund names and share classes including trademark symbols

There are many fund names and share classes have trademark symbols ™ or registerd trademark symbol ®. Some of them are missing in certain occassions while presenting in other ones. Sometimes, even though they present in all records, their print out are different. Therefore, we need to clean the fund and share class names in order to match the data.

In [16]:
# find the fund and share class names that contains trademark symbols

tm_shareClasses = shareclasses.str.contains('™')
tm_fundNames = fund_names.str.contains('™')

r_shareClasses = shareclasses.str.contains('®')
r_fundNames = fund_names.str.contains('®')

In [17]:
shareClasses_tm = set(shareclasses[tm_shareClasses])
fundNames_tm = set(fund_names[tm_fundNames])
                                 
shareClasses_r = set(shareclasses[r_shareClasses])
fundNames_r = set(fund_names[r_fundNames])

In [18]:
len(shareClasses_tm)

33

In [19]:
len(fundNames_tm)

36

In [20]:
len(shareClasses_r)

1406

In [21]:
len(fundNames_r)

665

In [22]:
list(shareClasses_tm)[:5]

['Vanguard Windsor™ II Admiral™',
 'Invesco Cleantech™ ETF',
 'Schwab International Small-Cap Eq ETF™',
 'Invesco RAFI™StrategicDvlpdex-USSmComETF',
 'Schwab US Broad Market ETF™']

In [23]:
list(fundNames_tm)[:5]

['Invesco Cleantech™ ETF',
 'Schwab Global Real Estate Fund™',
 'Vanguard Windsor™ Fund',
 'Invesco High Yield Equity Dividend Achievers™ ETF',
 'Invesco RAFI™ Strategic US Small Company ETF']

In [24]:
list(fundNames_r)[25:35]

['Fidelity® SAI U.S. Large Cap Index Fund',
 'Fidelity® SAI International Value Index Fund',
 'Fidelity® Emerging Markets Multifactor ETF',
 'Fidelity® Select Gold Portfolio',
 'Fidelity® US Sustainability Index Fund',
 'BlackRock LifePath® Index 2035 Fund',
 'Fidelity® Balanced Fund',
 'SPDR®\xa0Dow Jones\xa0REIT ETF',
 'VanEck® Green Infrastructure ETF',
 'SPDR® Solactive Japan ETF']

In [25]:
list(shareClasses_r)[30:40]

['Fidelity® Advisor Emerging Markets M',
 'Fidelity Advisor® Total Emerg Mkts A',
 'Fidelity® Real Estate Investment Port',
 'Fidelity® Equity Dividend Income',
 'Fidelity® International Index',
 'JPMorgan SmartRetirement® 2060 A',
 'Fidelity Advisor Asset Manager® 50% I',
 'JPMorgan SmartRetirement® Blend 2065 R6',
 "Fidelity Advisor® Women's Leadership Z",
 'Fidelity® SAI US Quality Index']

In [26]:
# Here is an example that the same string can be printed out differently

ex_fundNames = fund_names.str.contains('SPDR®')
list(set(fund_names[ex_fundNames]))[:10]

['SPDR® MSCI Emerging Markets Fossil Fuel Free ETF',
 'SPDR® S&P 500 Fossil Fuel Reserves Free ETF',
 'SPDR®\xa0S&P\xa0600 Small Cap Value ETF',
 'SPDR® Portfolio Europe ETF',
 'SPDR® S&P MIDCAP 400 ETF Trust',
 'Communication Services Select Sector SPDR® Fund',
 'SPDR®\xa0Global Dow ETF',
 'SPDR®\xa0S&P\xa0600 Small Cap Growth ETF',
 'SPDR® S&P Kensho Smart Mobility ETF',
 'SPDR® Russell 1000® Yield Focus ETF']

In [27]:
# We can firstly replace the '\xa0' string by ' '

fund_names_clean = fund_names.str.replace('\xa0', ' ')
list(set(fund_names_clean[ex_fundNames]))[:10]

['SPDR® MSCI Emerging Markets Fossil Fuel Free ETF',
 'SPDR® S&P 500 Fossil Fuel Reserves Free ETF',
 'SPDR® S&P 1500 Value Tilt ETF',
 'SPDR® Portfolio Europe ETF',
 'SPDR® S&P MIDCAP 400 ETF Trust',
 'Communication Services Select Sector SPDR® Fund',
 'Industrial Select Sector SPDR® Fund',
 'SPDR® MSCI World StrategicFactors ETF',
 'Technology Select Sector SPDR® Fund',
 'Consumer Discretionary Select Sector SPDR® Fund']

In [28]:
shareclass_clean = shareclasses.str.replace('\xa0', ' ')
list(set(shareclass_clean[shareclasses.str.contains('SPDR®')]))[:10]

['SPDR® S&P 1500 Value Tilt ETF',
 'SPDR® Portfolio Europe ETF',
 'SPDR® S&P MIDCAP 400 ETF Trust',
 'SPDR® S&P 500 Fossil Fuel Rsrv Free ETF',
 'SPDR® Portfolio Developed Wld ex-US ETF',
 'SPDR® MSCI EAFE Fssl Ful Free RsrvETF',
 'SPDR® MSCI World StrategicFactors ETF',
 'SPDR® SSGA US Small Cap Low Volatil ETF',
 'SPDR® S&P Kensho Smart Mobility ETF',
 'SPDR® S&P International Small Cap ETF']

In [29]:
# The trade mark symbol is missing in certain records
list(set(fund_names_clean[fund_names_clean.str.contains('SPDR ')]))[:]

['SPDR Bloomberg SASB Developed Markets Ex US ESG Select ETF',
 'SPDR S&P® North American Natural Resources ETF',
 'SPDR S&P SmallCap 600 ESG ETF',
 'The Real Estate Select Sector SPDR Fund',
 'SPDR Bloomberg SASB Emerging Markets ESG Select ETF',
 'SPDR MSCI USA Climate Paris Aligned ETF']

For the data matching purpose, it is necessary to remove all the trade mark symbols and keep an space between words but eliminate head or trailing spaces.

In [30]:
# copy the original columns of fund names and share class names

df['Shareclass name'] = df['Fund profile: Shareclass name']
df['Fund name'] = df['Fund profile: Fund name']

In [31]:
# clean the new columns by replacing the '\xa0' string by ' '

df['Shareclass name'] = df['Shareclass name'].str.replace('\xa0', ' ')
df['Fund name'] = df['Fund name'].str.replace('\xa0', ' ')

In [32]:
# Also drop the trademark symbols (™ and ®)
# note that there might be space before and/or after the symbols

def replace_trademark(s):
    s = s.replace('™', ' ')
    s = re.sub('®', ' ', s)
    s = re.sub('\s+', ' ', s)
    return s.strip()
    
df['Shareclass name'] = df['Shareclass name'].apply(replace_trademark)
df['Fund name'] = df['Fund name'].apply(replace_trademark)

In [33]:
# examine the results

df[['Fund profile: Shareclass name','Shareclass name',
    'Fund profile: Fund name','Fund name']][df['Fund profile: Fund name'].str.contains('\xa0')].head()


,Fund profile: Shareclass name,Shareclass name,Fund profile: Fund name,Fund name
5570,JNL/RAFI® Fundamental Asia Developed A,JNL/RAFI Fundamental Asia Developed A,JNL/RAFI® Fundamental Asia Developed Fund,JNL/RAFI Fundamental Asia Developed Fund
5571,JNL/RAFI® Fundamental Asia Developed I,JNL/RAFI Fundamental Asia Developed I,JNL/RAFI® Fundamental Asia Developed Fund,JNL/RAFI Fundamental Asia Developed Fund
7734,Consumer Discret Sel Sect SPDR® ETF,Consumer Discret Sel Sect SPDR ETF,Consumer Discretionary Select Sector SPDR® Fund,Consumer Discretionary Select Sector SPDR Fund
7736,Energy Select Sector SPDR® ETF,Energy Select Sector SPDR ETF,Energy Select Sector SPDR® Fund,Energy Select Sector SPDR Fund
7737,Financial Select Sector SPDR® ETF,Financial Select Sector SPDR ETF,Financial Select Sector SPDR® Fund,Financial Select Sector SPDR Fund


In [34]:
df[['Fund profile: Shareclass name','Shareclass name','Fund profile: Fund name','Fund name']][df['Fund profile: Fund name'].str.contains('™')].head()


,Fund profile: Shareclass name,Shareclass name,Fund profile: Fund name,Fund name
2426,Eaton Vance Global Income Builder NS™,Eaton Vance Global Income Builder NS,Eaton Vance Global Income Builder NextShares™,Eaton Vance Global Income Builder NextShares
2464,Eaton Vance Stock NextShares™,Eaton Vance Stock NextShares,Eaton Vance Stock NextShares™,Eaton Vance Stock NextShares
4395,Invesco Cleantech™ ETF,Invesco Cleantech ETF,Invesco Cleantech™ ETF,Invesco Cleantech ETF
4433,Invesco Dividend Achievers™ ETF,Invesco Dividend Achievers ETF,Invesco Dividend Achievers™ ETF,Invesco Dividend Achievers ETF
4577,Invesco High Yield Eq Div Achiev™ ETF,Invesco High Yield Eq Div Achiev ETF,Invesco High Yield Equity Dividend Achievers™ ETF,Invesco High Yield Equity Dividend Achievers ETF


In [35]:
df[['Fund profile: Shareclass name','Shareclass name',
    'Fund profile: Fund name','Fund name']][df['Fund profile: Fund name'].str.contains('®')].head()


,Fund profile: Shareclass name,Shareclass name,Fund profile: Fund name,Fund name
507,American Century Ultra® A,American Century Ultra A,American Century Ultra® Fund,American Century Ultra Fund
508,American Century Ultra® C,American Century Ultra C,American Century Ultra® Fund,American Century Ultra Fund
509,American Century Ultra® G,American Century Ultra G,American Century Ultra® Fund,American Century Ultra Fund
510,American Century Ultra® I,American Century Ultra I,American Century Ultra® Fund,American Century Ultra Fund
511,American Century Ultra® Inv,American Century Ultra Inv,American Century Ultra® Fund,American Century Ultra Fund


### 2.3 Merge duplicated columns with different names

There are several columns that are duplicated in meaning. For example, 'Returns and fees: Month end trailing returns, 1 year' and 'Financial performance: Month end trailing returns, year 1'. They are the same variable with different names in the original files. We should merge them to a single column.

In [36]:
key_str = 'Financial performance:'
sub_col = df.filter(regex = key_str)
sub_col.columns

Index(['Financial performance: Financial performance as-of date',
       'Financial performance: Month end trailing returns, year 1',
       'Financial performance: Month end trailing returns, year 3',
       'Financial performance: Month end trailing returns, year 5',
       'Financial performance: Month end trailing returns, year 10'],
      dtype='object')

In [37]:
key_str2 = 'Returns and fees:'
sub_col2 = df.filter(regex = key_str2)
sub_col2.columns

Index(['Returns and fees: Financial performance as-of date',
       'Returns and fees: Month end trailing returns, 1 month',
       'Returns and fees: Month end trailing returns, 3 month',
       'Returns and fees: Month end trailing returns, 6 month',
       'Returns and fees: Month end trailing returns, 1 year',
       'Returns and fees: Month end trailing returns, 3 year',
       'Returns and fees: Month end trailing returns, 5 year',
       'Returns and fees: Month end trailing returns, 10 year',
       'Returns and fees: Month end trailing returns, 15 year',
       'Returns and fees: Month end trailing returns, 20 year',
       'Returns and fees: Month end trailing returns, year-to-date',
       'Returns and fees: Month end trailing returns, since inception',
       'Returns and fees: Prospectus net expense ratio'],
      dtype='object')

In [38]:
# merge the duplicated columns for 'Month end trailing returns, 1 year' to a new column

df['Month end trailing returns, 1 year']= (df['Financial performance: Month end trailing returns, year 1']
                                           .combine_first(df['Returns and fees: Month end trailing returns, 1 year']))

In [39]:
Non_missing = df['Month end trailing returns, 1 year'].count()

In [40]:
Non_missing1 = df['Returns and fees: Month end trailing returns, 1 year'].count()

In [41]:
Non_missing2 = df['Financial performance: Month end trailing returns, year 1'].count()

In [42]:
assert Non_missing == Non_missing1 + Non_missing2

In [43]:
# merge the other duplicated columns

df['Month end trailing returns, 3 year']= (df['Financial performance: Month end trailing returns, year 3']
                                           .combine_first(df['Returns and fees: Month end trailing returns, 3 year']))
df['Month end trailing returns, 5 year']= (df['Financial performance: Month end trailing returns, year 5']
                                           .combine_first(df['Returns and fees: Month end trailing returns, 5 year']))
df['Month end trailing returns, 10 year']= (df['Financial performance: Month end trailing returns, year 10']
                                            .combine_first(df['Returns and fees: Month end trailing returns, 10 year']))
df['Financial performance as-of date']= (df['Financial performance: Financial performance as-of date']
                                         .combine_first(df['Returns and fees: Financial performance as-of date']))


In [44]:
df.head()

,Fund profile: Shareclass name,Fund profile: Ticker,Fund profile: Fund name,Fund profile: Asset manager,Fund profile: Shareclass type,Fund profile: Shareclass inception date,Fund profile: Category group,Fund profile: Sustainability mandate,Fund profile: US-SIF member,Fund profile: Oldest shareclass inception date,...,"Fossil Free Funds: Fossil fuel insurance holdings, count","Fossil Free Funds: Fossil fuel insurance holdings, weight","Fossil Free Funds: Fossil fuel insurance holdings, asset",Shareclass name,Fund name,"Month end trailing returns, 1 year","Month end trailing returns, 3 year","Month end trailing returns, 5 year","Month end trailing returns, 10 year",Financial performance as-of date
0,1290 SmartBeta Equity A,TNBRX,1290 SmartBeta Equity Fund,1290 Funds,Open-end mutual fund,2014-11-12,International Equity Funds,Y,NaN,2014-11-12,...,NaN,NaN,NaN,1290 SmartBeta Equity A,1290 SmartBeta Equity Fund,-7.74546,3.45056,4.32708,NaN,2020-03-31
1,1290 SmartBeta Equity I,TNBRX,1290 SmartBeta Equity Fund,1290 Funds,Open-end mutual fund,2014-11-12,International Equity Funds,Y,NaN,2014-11-12,...,NaN,NaN,NaN,1290 SmartBeta Equity I,1290 SmartBeta Equity Fund,-7.50301,3.69803,4.58249,NaN,2020-03-31
2,1290 SmartBeta Equity R,TNBRX,1290 SmartBeta Equity Fund,1290 Funds,Open-end mutual fund,2014-11-12,International Equity Funds,Y,NaN,2014-11-12,...,NaN,NaN,NaN,1290 SmartBeta Equity R,1290 SmartBeta Equity Fund,-7.97413,3.16886,4.04976,NaN,2020-03-31
3,1290 SmartBeta Equity T,TNBRX,1290 SmartBeta Equity Fund,1290 Funds,Open-end mutual fund,2014-11-12,International Equity Funds,Y,NaN,2014-11-12,...,NaN,NaN,NaN,1290 SmartBeta Equity T,1290 SmartBeta Equity Fund,-7.52837,3.68855,4.57675,NaN,2020-03-31
4,13D Activist A,DDDCX,13D Activist Fund,13D Activist Fund,Open-end mutual fund,2011-12-28,U.S. Equity Fund,Y,NaN,2011-12-28,...,NaN,NaN,NaN,13D Activist A,13D Activist Fund,-23.01826,-3.84150,-1.04910,NaN,2020-03-31


In [45]:
# drop the incomplete columns used to merge to complete ones

df = df.drop(['Financial performance: Month end trailing returns, year 1',
             'Returns and fees: Month end trailing returns, 1 year',
             'Financial performance: Month end trailing returns, year 3',
             'Returns and fees: Month end trailing returns, 3 year',
             'Financial performance: Month end trailing returns, year 5',
             'Returns and fees: Month end trailing returns, 5 year',
             'Financial performance: Month end trailing returns, year 10',
             'Returns and fees: Month end trailing returns, 10 year',
             'Financial performance: Financial performance as-of date',
             'Returns and fees: Financial performance as-of date'], 1)

/var/folders/7m/7twstpk15s19tzfmdbvk7v680000gn/T/ipykernel_29005/3789892131.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['Financial performance: Month end trailing returns, year 1',


### 2.4 Deal with the double reported months

Some months' performance are reported in two consecutive months' data. Are these double reported data duplicated?

In [46]:
# 'current time' was added when loading data to differentiate batches of data
# it is the reporting time

time_table = df[['Financial performance as-of date','current time']]
tt = time_table.drop_duplicates()
tt.head()

,Financial performance as-of date,current time
0,2020-03-31,20200414
9238,2020-04-30,20200519
13992,2020-03-31,20200519
18476,2020-05-31,20200610
20686,2020-04-30,20200610


In [47]:
tt.sort_values(['Financial performance as-of date', 'current time']).head(31)

,Financial performance as-of date,current time
0,2020-03-31,20200414
13992,2020-03-31,20200519
9238,2020-04-30,20200519
20686,2020-04-30,20200610
18476,2020-05-31,20200610
27861,2020-05-31,20200716
27846,2020-06-30,20200716
37183,2020-07-31,20200811
46423,2020-08-31,20200913
55648,2020-08-31,20200928


In [48]:
# the two largest counts of records indicate the double reporting for the same date

df['Financial performance as-of date'].value_counts().head()

2020-08-31    18450
2022-01-31    17573
2020-05-31     9384
2020-06-30     9316
2020-07-31     9240
Name: Financial performance as-of date, dtype: int64

In [49]:
df['current time'].value_counts().head()

20200610    9370
20200716    9337
20200811    9240
20200414    9238
20200519    9238
Name: current time, dtype: int64

In [50]:
col_names = df.columns
type(col_names)

pandas.core.indexes.base.Index

In [51]:
len(col_names)

143

In [52]:
# 'current time' should be ignored when judging duplicated rows

col_names_subset = col_names.drop('current time')

In [53]:
len(col_names_subset)

142

In [54]:
duplicated_rows = df.duplicated(subset = col_names_subset)

In [55]:
# number of duplicated rows in the data

sum(duplicated_rows)

7092

In [56]:
df[duplicated_rows].head()

,Fund profile: Shareclass name,Fund profile: Ticker,Fund profile: Fund name,Fund profile: Asset manager,Fund profile: Shareclass type,Fund profile: Shareclass inception date,Fund profile: Category group,Fund profile: Sustainability mandate,Fund profile: US-SIF member,Fund profile: Oldest shareclass inception date,...,"Fossil Free Funds: Fossil fuel insurance holdings, count","Fossil Free Funds: Fossil fuel insurance holdings, weight","Fossil Free Funds: Fossil fuel insurance holdings, asset",Shareclass name,Fund name,"Month end trailing returns, 1 year","Month end trailing returns, 3 year","Month end trailing returns, 5 year","Month end trailing returns, 10 year",Financial performance as-of date
200365,ALPS/Kotak India Growth A,INAAX,ALPS/Kotak India Growth,ALPS,Open-end mutual fund,2018-06-12,International Equity Funds,Y,NaN,2011-02-14,...,NaN,NaN,NaN,ALPS/Kotak India Growth A,ALPS/Kotak India Growth,25.55341,16.95053,12.37844,11.83715,2022-01-31
200366,ALPS/Kotak India Growth C,INFCX,ALPS/Kotak India Growth,ALPS,Open-end mutual fund,2011-02-14,International Equity Funds,Y,NaN,2011-02-14,...,NaN,NaN,NaN,ALPS/Kotak India Growth C,ALPS/Kotak India Growth,24.69715,16.11809,11.57407,11.01872,2022-01-31
200367,ALPS/Kotak India Growth I,INDIX,ALPS/Kotak India Growth,ALPS,Open-end mutual fund,2011-02-14,International Equity Funds,Y,NaN,2011-02-14,...,NaN,NaN,NaN,ALPS/Kotak India Growth I,ALPS/Kotak India Growth,25.93037,17.26608,12.69606,12.13499,2022-01-31
200368,ALPS/Kotak India Growth II,NaN,ALPS/Kotak India Growth,ALPS,Open-end mutual fund,2019-12-19,International Equity Funds,Y,NaN,2011-02-14,...,NaN,NaN,NaN,ALPS/Kotak India Growth II,ALPS/Kotak India Growth,26.29507,17.50607,12.83439,12.20379,2022-01-31
200369,ALPS/Kotak India Growth Inv,INDAX,ALPS/Kotak India Growth,ALPS,Open-end mutual fund,2011-02-14,International Equity Funds,Y,NaN,2011-02-14,...,NaN,NaN,NaN,ALPS/Kotak India Growth Inv,ALPS/Kotak India Growth,25.53898,17.09529,12.43900,11.81849,2022-01-31


In [57]:
# which date(s) have duplicated reporting

df.loc[duplicated_rows, 'Financial performance as-of date'].unique()

array(['2022-01-31', nan], dtype=object)

In [58]:
# drop these duplicated rows

df = df[~duplicated_rows]
df.shape

(349426, 143)

In [59]:
# some months performance reported in two consecutive months but not duplicated
# use the rows that was reported in the latest month

df = df.sort_values(['current time'])
df.drop_duplicates(subset = ['Shareclass name', 'Financial performance as-of date'], keep = 'last', inplace = True)
df.shape

(286537, 143)

In [60]:
row_counts = df.groupby(['Shareclass name', 'Financial performance as-of date']).apply(len)
sum(row_counts > 1)

0

### 2.5 Find the funds and share classes that have target variables over longest period

Firstly, we can choose the 'Month end trailing returns, 1 year' as our target variable

In [61]:
df_sub = pd.concat([df['Financial performance as-of date'], df['current time']], axis = 1)
df_sub.columns = ['date1', 'date2']
df_sub.head()

,date1,date2
0,2020-03-31,20200414
6155,2020-03-31,20200414
6156,2020-03-31,20200414
6157,2020-03-31,20200414
6158,2020-03-31,20200414


In [62]:
df_sub.describe()

,date1,date2
count,277579,286537
unique,34,39
top,2020-05-31,20200610
freq,9363,9349


In [63]:
time_series_data = df[['Shareclass name','Financial performance as-of date','Month end trailing returns, 1 year']]


In [64]:
time_series_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286537 entries, 0 to 356517
Data columns (total 3 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Shareclass name                     286537 non-null  object 
 1   Financial performance as-of date    277579 non-null  object 
 2   Month end trailing returns, 1 year  271807 non-null  float64
dtypes: float64(1), object(2)
memory usage: 8.7+ MB


In [65]:
sorted_data = time_series_data.sort_values(['Shareclass name','Financial performance as-of date'])

In [66]:
sorted_data.head(16)

,Shareclass name,Financial performance as-of date,"Month end trailing returns, 1 year"
0,1290 SmartBeta Equity A,2020-03-31,-7.74546
9238,1290 SmartBeta Equity A,2020-04-30,-2.73874
18476,1290 SmartBeta Equity A,2020-05-31,5.15167
27846,1290 SmartBeta Equity A,2020-06-30,0.92022
37183,1290 SmartBeta Equity A,2020-07-31,5.48843
55648,1290 SmartBeta Equity A,2020-08-31,10.96411
64873,1290 SmartBeta Equity A,2020-09-30,6.55784
1,1290 SmartBeta Equity I,2020-03-31,-7.50301
9239,1290 SmartBeta Equity I,2020-04-30,-2.46099
18477,1290 SmartBeta Equity I,2020-05-31,5.38630


In [67]:
sorted_data.shape

(286537, 3)

In [68]:
sorted_data['Shareclass name'].nunique()

14046

In [69]:
non_missing_data = sorted_data.dropna(subset = ['Financial performance as-of date', 
                                                'Month end trailing returns, 1 year'])
non_missing_data.shape

(271807, 3)

In [70]:
freq_counts = non_missing_data['Shareclass name'].value_counts()
sorted_counts = freq_counts.sort_values(ascending = False).reset_index()
sorted_counts.columns = ['Shareclass name', '# time steps']
sorted_counts.head()

,Shareclass name,# time steps
0,Goldman Sachs International Eq Inc Inv,31
1,Janus Henderson Growth And Income S,31
2,Janus Henderson Overseas C,31
3,Janus Henderson Overseas D,31
4,Janus Henderson Overseas I,31


In [71]:
max_t = sorted_counts['# time steps'].max()

In [72]:
sorted_counts.shape

(13438, 2)

In [73]:
shareclasses_sub = sorted_counts.loc[sorted_counts['# time steps'] == max_t,'Shareclass name']
shareclasses_sub.head()

0    Goldman Sachs International Eq Inc Inv
1       Janus Henderson Growth And Income S
2                Janus Henderson Overseas C
3                Janus Henderson Overseas D
4                Janus Henderson Overseas I
Name: Shareclass name, dtype: object

In [74]:
shareclasses_list = list(shareclasses_sub)
len(shareclasses_list)

5579

In [75]:
# the months are not all continuous. 
# some months are missing, in the first share class, 2020-11-30 and 2021-04-30 are missing
# also for last month, 2022-12-31 is NAN

longest_ts = sorted_data[sorted_data['Shareclass name'].isin(shareclasses_list)]
longest_ts.head(33)

,Shareclass name,Financial performance as-of date,"Month end trailing returns, 1 year"
7,1919 Socially Responsive Balanced A,2020-03-31,0.00352
9245,1919 Socially Responsive Balanced A,2020-04-30,5.60674
18483,1919 Socially Responsive Balanced A,2020-05-31,13.11467
27853,1919 Socially Responsive Balanced A,2020-06-30,10.91060
37190,1919 Socially Responsive Balanced A,2020-07-31,15.01209
55655,1919 Socially Responsive Balanced A,2020-08-31,21.47144
64880,1919 Socially Responsive Balanced A,2020-09-30,18.34213
74102,1919 Socially Responsive Balanced A,2020-10-31,15.15362
83172,1919 Socially Responsive Balanced A,2020-12-31,20.57031
92372,1919 Socially Responsive Balanced A,2021-01-31,17.88592


In [76]:
# check if these two time step is missing for all share classes

check_missing = longest_ts.loc[longest_ts['Financial performance as-of date'].isin(['2020-11-30', '2021-04-30'])]
check_missing

,Shareclass name,Financial performance as-of date,"Month end trailing returns, 1 year"


None of the share classes with the longest time series of 'Month end trailing returns, 1 year' have this performance reported for these two months.

In [77]:
# drop the NaN values

longest_ts = longest_ts.loc[~longest_ts['Financial performance as-of date'].isnull()]
longest_ts.head(33)

,Shareclass name,Financial performance as-of date,"Month end trailing returns, 1 year"
7,1919 Socially Responsive Balanced A,2020-03-31,0.00352
9245,1919 Socially Responsive Balanced A,2020-04-30,5.60674
18483,1919 Socially Responsive Balanced A,2020-05-31,13.11467
27853,1919 Socially Responsive Balanced A,2020-06-30,10.91060
37190,1919 Socially Responsive Balanced A,2020-07-31,15.01209
55655,1919 Socially Responsive Balanced A,2020-08-31,21.47144
64880,1919 Socially Responsive Balanced A,2020-09-30,18.34213
74102,1919 Socially Responsive Balanced A,2020-10-31,15.15362
83172,1919 Socially Responsive Balanced A,2020-12-31,20.57031
92372,1919 Socially Responsive Balanced A,2021-01-31,17.88592


### 2.6 Save the time series data and the data records with target covering longest time period

In [78]:
# save the time series data

longest_ts.to_csv('data/shareclasses_one_year_return_max_months_long_time_series.csv', index = False)

In [79]:
# include the features and other targets

df_filtered = df[df['Shareclass name'].isin(shareclasses_list)]
df_filtered.shape

(178505, 143)

In [80]:
df_filtered['Shareclass name'].nunique()

5579

In [81]:
df_filtered.tail()

,Fund profile: Shareclass name,Fund profile: Ticker,Fund profile: Fund name,Fund profile: Asset manager,Fund profile: Shareclass type,Fund profile: Shareclass inception date,Fund profile: Category group,Fund profile: Sustainability mandate,Fund profile: US-SIF member,Fund profile: Oldest shareclass inception date,...,"Fossil Free Funds: Fossil fuel insurance holdings, count","Fossil Free Funds: Fossil fuel insurance holdings, weight","Fossil Free Funds: Fossil fuel insurance holdings, asset",Shareclass name,Fund name,"Month end trailing returns, 1 year","Month end trailing returns, 3 year","Month end trailing returns, 5 year","Month end trailing returns, 10 year",Financial performance as-of date
350699,Fidelity Advisor Asset Manager® 60% A,FSAAX,Fidelity Asset Manager® 60% Fund,Fidelity Investments,Open-end mutual fund,2007-10-09,Allocation Funds,NaN,NaN,2007-10-09,...,9.0,0.010554,41565360.0,Fidelity Advisor Asset Manager 60% A,Fidelity Asset Manager 60% Fund,NaN,NaN,NaN,NaN,NaN
350698,Fidelity® Value Strategies,FSLSX,Fidelity Advisor® Value Strategies Fund,Fidelity Investments,Open-end mutual fund,1983-12-30,U.S. Equity Fund,NaN,NaN,1983-12-30,...,1.0,0.013164,20397790.0,Fidelity Value Strategies,Fidelity Advisor Value Strategies Fund,NaN,NaN,NaN,NaN,NaN
350697,Fidelity Advisor® Value Strategies M,FASPX,Fidelity Advisor® Value Strategies Fund,Fidelity Investments,Open-end mutual fund,1986-08-20,U.S. Equity Fund,NaN,NaN,1983-12-30,...,1.0,0.013164,20397790.0,Fidelity Advisor Value Strategies M,Fidelity Advisor Value Strategies Fund,NaN,NaN,NaN,NaN,NaN
350696,Fidelity Advisor® Value Strategies K,FVSKX,Fidelity Advisor® Value Strategies Fund,Fidelity Investments,Open-end mutual fund,2008-05-09,U.S. Equity Fund,NaN,NaN,1983-12-30,...,1.0,0.013164,20397790.0,Fidelity Advisor Value Strategies K,Fidelity Advisor Value Strategies Fund,NaN,NaN,NaN,NaN,NaN
350695,Fidelity Advisor® Value Strategies I,FASOX,Fidelity Advisor® Value Strategies Fund,Fidelity Investments,Open-end mutual fund,1995-07-03,U.S. Equity Fund,NaN,NaN,1983-12-30,...,1.0,0.013164,20397790.0,Fidelity Advisor Value Strategies I,Fidelity Advisor Value Strategies Fund,NaN,NaN,NaN,NaN,NaN


In [82]:
df_filtered.to_csv('data/shareclasses_one_year_return_max_months_long_full_data.csv', index = False)

In [83]:
# save dtype of each column
datatypes2  = df_filtered.dtypes.to_frame('dtypes').reset_index()
type(datatypes2)

pandas.core.frame.DataFrame

In [84]:
datatypes2.tail()

,index,dtypes
138,"Month end trailing returns, 1 year",float64
139,"Month end trailing returns, 3 year",float64
140,"Month end trailing returns, 5 year",float64
141,"Month end trailing returns, 10 year",float64
142,Financial performance as-of date,object


In [85]:
datatypes2.to_csv('data/shareclasses_one_year_return_max_months_long_full_data_datatypes.csv', index = False)

## 3. Data Definition

In this section, the target variables and feature variables will be defined. The feature variables are grouped by the categories. 

### 3.1 Load the data

In [128]:
# Load the saved data

types2 = pd.read_csv('data/shareclasses_one_year_return_max_months_long_full_data_datatypes.csv')
dtypes_dict2 = types2.set_index('index')['dtypes'].to_dict()

df_1yr = pd.read_csv('data/shareclasses_one_year_return_max_months_long_full_data.csv', dtype = dtypes_dict2)

### 3.2 Define the target

Although the data were selected based on the month end trailing returns of one year, there are other target variables available in the dataframe. 

In [129]:
target_cols = [col for col in df_1yr.columns if 'returns' in col or 'Returns' in col]
target_cols

['Returns and fees: Month end trailing returns, 1 month',
 'Returns and fees: Month end trailing returns, 3 month',
 'Returns and fees: Month end trailing returns, 6 month',
 'Returns and fees: Month end trailing returns, 15 year',
 'Returns and fees: Month end trailing returns, 20 year',
 'Returns and fees: Month end trailing returns, year-to-date',
 'Returns and fees: Month end trailing returns, since inception',
 'Returns and fees: Prospectus net expense ratio',
 'Month end trailing returns, 1 year',
 'Month end trailing returns, 3 year',
 'Month end trailing returns, 5 year',
 'Month end trailing returns, 10 year']

In [130]:
# Check how many missing data in each target

df_1yr[target_cols].isnull().mean().sort_values()

Month end trailing returns, 1 year                               0.031125
Month end trailing returns, 3 year                               0.044940
Month end trailing returns, 5 year                               0.089885
Month end trailing returns, 10 year                              0.210661
Returns and fees: Prospectus net expense ratio                   0.374919
Returns and fees: Month end trailing returns, year-to-date       0.375211
Returns and fees: Month end trailing returns, 6 month            0.375250
Returns and fees: Month end trailing returns, since inception    0.375872
Returns and fees: Month end trailing returns, 3 month            0.375889
Returns and fees: Month end trailing returns, 1 month            0.376292
Returns and fees: Month end trailing returns, 15 year            0.569239
Returns and fees: Month end trailing returns, 20 year            0.650508
dtype: float64

The column 'Month end trailing returns, 1 year' will be our main target as it has the least missing data

### 3.3 Define the features

In [131]:
column_names = df_1yr.columns
column_names[120:]

Index(['Returns and fees: Month end trailing returns, 15 year',
       'Returns and fees: Month end trailing returns, 20 year',
       'Returns and fees: Month end trailing returns, year-to-date',
       'Returns and fees: Month end trailing returns, since inception',
       'Returns and fees: Prospectus net expense ratio',
       'Fund profile: Target date',
       'Fossil Free Funds: Fossil fuel finance grade',
       'Fossil Free Funds: Fossil fuel finance fund score',
       'Fossil Free Funds: Fossil fuel finance holdings, count',
       'Fossil Free Funds: Fossil fuel finance holdings, weight',
       'Fossil Free Funds: Fossil fuel finance holdings, asset',
       'Fossil Free Funds: Fossil fuel insurance grade',
       'Fossil Free Funds: Fossil fuel insurance fund score',
       'Fossil Free Funds: Fossil fuel insurance holdings, count',
       'Fossil Free Funds: Fossil fuel insurance holdings, weight',
       'Fossil Free Funds: Fossil fuel insurance holdings, asset',
      

In [132]:
# There are several categories of feature variables

cat_features = set([colname.split(':')[0] for colname in column_names 
                    if ':' in colname and 'Returns and fees' not in colname])
cat_features

{'Deforestation Free Funds',
 'Fossil Free Funds',
 'Fund profile',
 'Gender Equality Funds',
 'Gun Free Funds',
 'Prison Free Funds',
 'Tobacco Free Funds',
 'Weapon Free Funds'}

In [133]:
# The feature category of most interest of this project is 'Fossil Free Funds'

df_1yr.filter(like = 'Fossil Free Funds: ').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178505 entries, 0 to 178504
Data columns (total 38 columns):
 #   Column                                                                                 Non-Null Count   Dtype  
---  ------                                                                                 --------------   -----  
 0   Fossil Free Funds: Fossil fuel grade                                                   178500 non-null  object 
 1   Fossil Free Funds: Fossil fuel holdings, count                                         178505 non-null  int64  
 2   Fossil Free Funds: Fossil fuel holdings, weight                                        178505 non-null  float64
 3   Fossil Free Funds: Fossil fuel holdings, asset                                         178505 non-null  int64  
 4   Fossil Free Funds: Carbon Underground 200, count                                       178505 non-null  int64  
 5   Fossil Free Funds: Carbon Underground 200, weight                

In [134]:
# In this category, 'Fossil fuel grade' is one of those with the lease missing values

df_grade = df_1yr['Fossil Free Funds: Fossil fuel grade']
df_grade.head()

0    F
1    F
2    F
3    F
4    F
Name: Fossil Free Funds: Fossil fuel grade, dtype: object

In [135]:
# combine the fossil fuel grade column with the one year return target column

df_return = df_1yr['Month end trailing returns, 1 year']

df_sub = pd.concat([df_grade, df_return], axis = 1)
df_sub.columns = ['grade', 'return']
df_sub.head()

,grade,return
0,F,-13.18155
1,F,-13.85034
2,F,-12.95811
3,F,-12.90129
4,F,-13.54326


In [136]:
# see if higher grade corresponds to higher return

pivot_table = pd.pivot_table(df_sub, values = 'return', index = 'grade')
pivot_table

,return
grade,
A,14.602620
B,14.097770
C,9.107015
D,9.079069
F,7.257575


## 4. Data Cleaning

In [137]:
# check if there are duplicated rows

df_1yr.duplicated().sum()

0

In [138]:
# check if all rows have values for 'Month end trailing returns, 1 year', the main target variable

df_1yr['Month end trailing returns, 1 year'].isnull().mean()

0.03112517856642671

In [139]:
# check current dimension of the dataframe

df_1yr.shape

(178505, 143)

In [140]:
# drop the rows that do not have values for the main target

df_1yr = df_1yr.dropna(subset = 'Month end trailing returns, 1 year')
df_1yr.shape

(172949, 143)

In [141]:
# some features and targets have too many missing data
# Check how many missing data in each target and feature variables

missing_rate = df_1yr.isnull().mean().sort_values()
missing_rate[missing_rate != 0.0]

Fossil Free Funds: Fossil fuel grade                                                                              0.000029
Month end trailing returns, 3 year                                                                                0.014259
Fund profile: Shareclass tickers                                                                                  0.016490
Fund profile: Ticker                                                                                              0.019728
Gender Equality Funds: Gender equality group ranking                                                              0.050356
Gender Equality Funds: Gender equality grade                                                                      0.052750
Month end trailing returns, 5 year                                                                                0.060648
Prison Free Funds: Private prison operators, weight                                                               0.129032
Prison Free Fund

In [142]:
# drop columns with too many missing data 
df_1yr = df_1yr.loc[:, df_1yr.isnull().mean() < .1]
df_1yr.shape

(172949, 91)

In [143]:
# Save the data
df_1yr.to_csv('data/shareclasses_one_year_return_max_months_less_than_10pct_missing.csv', index = False)

In [144]:
# save dtype of each column
datatypes_1yr  = df_1yr.dtypes.to_frame('dtypes').reset_index()
datatypes_1yr.tail()

,index,dtypes
86,Fund name,object
87,"Month end trailing returns, 1 year",float64
88,"Month end trailing returns, 3 year",float64
89,"Month end trailing returns, 5 year",float64
90,Financial performance as-of date,object


In [145]:
datatypes_1yr.to_csv('data/shareclasses_one_year_return_max_months_less_than_10pct_missing_datatypes.csv',
                     index = False)

After the data wrangling, we have the following data tables:

1. the full raw data set: 
    'all_results_data.csv'
2. time series data of the shareclasses with the longest record (31 months) of the main target (Month end trailing returns, 1 year): 
    'shareclasses_one_year_return_max_months_long_time_series.csv'
3. the full cleaned data set of the shareclasses with the longest record of the main target including all features variables and target variables:
    'shareclasses_one_year_return_max_months_long_full_data.csv'
4. the subset of targets and features that have less than 10 percent of missing data for all the shareclasses with the longest record of the main target:
    'shareclasses_one_year_return_max_months_less_than_10pct_missing.csv'